https://towardsdatascience.com/step-by-step-twitter-sentiment-analysis-in-python-d6f650ade58d

In [1]:
# Install Libraries
!pip install textblob
!pip install tweepy
!pip install pycountry
!pip install langdetect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 10.1 MB 8.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pycountry: filename=pycountry-22.3.5-py2.py3-none-any.whl size=10681845 sha256=f40a845588e8bd70a1dfec850bcf7bdf8830953a9fa6e49f61e2144a4a06c6cf
  Stored in directory: /root/.cache/pip/wheels/0e/06/e8/7ee176e95ea9a8a8c3b3afcb1869f20adbd42413d4611c6eb4
Successfully built pycountry
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 981 kB 6.5 MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-n

In [2]:
# Import Libraries
from textblob import TextBlob
import sys
import tweepy
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import nltk
import pycountry
import re
import string
from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from langdetect import detect
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
from sklearn.feature_extraction.text import CountVectorizer
from google.colab import drive

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [3]:
# Mount GDrive
drive.mount('/gdrive')


Mounted at /gdrive


In [4]:
os.chdir('/gdrive/MyDrive/Docencia/UV-ML')
!ls

Twitter_Extraction_V2.ipynb


In [5]:
# Authentication
consumerKey = ""
consumerSecret = ""
accessToken = ""
accessTokenSecret = ""
auth = tweepy.OAuthHandler(consumerKey, consumerSecret)
auth.set_access_token(accessToken, accessTokenSecret)
api = tweepy.API(auth)

In [14]:
#Sentiment Analysis
def percentage(part,whole):
 return 100 * float(part)/float(whole)
# parámetros query: https://medium.com/@robguilarr/making-queries-to-twitter-api-on-tweepy-66afeb7184a4
keyword = '"Universidad de Valencia" OR "Universitat de València"'
#keyword = "ropa_ de_segunda_mano OR ropa_casi_nueva"
noOfTweet = int(10)

#searchCountry = "Spain"
#places = api.geo_search(query=searchCountry, granularity="country")
#place_id = places[0].id

tweets = tweepy.Cursor(api.search, q=(keyword),lang='es'  ).items(noOfTweet)
positive = 0
negative = 0
neutral = 0
polarity = 0
tweet_list = []
neutral_list = []
negative_list = []
positive_list = []
date_list = []
score_list = []
user_list =[]
location_list =[]


In [15]:
for tweet in tweets:
 
  #print(tweet.text)
  tweet_list.append(tweet.text)
  analysis = TextBlob(tweet.text)
  score = SentimentIntensityAnalyzer().polarity_scores(tweet.text)
  date_list.append(tweet.created_at)
  score_list.append(score)
  user_list.append(tweet.user.name)
  location_list.append(tweet.user.location)

  neg = score['neg']
  neu = score['neu']
  pos = score['pos']
  comp = score['compound']
  polarity += analysis.sentiment.polarity
  
  if neg > pos:
    negative_list.append(tweet.text)
    negative += 1
  elif pos > neg:
    positive_list.append(tweet.text)
    positive += 1
  
  elif pos == neg:
    neutral_list.append(tweet.text)
    neutral += 1
positive = percentage(positive, noOfTweet)
negative = percentage(negative, noOfTweet)
neutral = percentage(neutral, noOfTweet)
polarity = percentage(polarity, noOfTweet)
positive = format(positive, '.1f')
negative = format(negative, '.1f')
neutral = format(neutral, '.1f')

In [16]:
#Number of Tweets (Total, Positive, Negative, Neutral)
tweet_list_2 = pd.DataFrame(tweet_list)
neutral_list = pd.DataFrame(neutral_list)
negative_list = pd.DataFrame(negative_list)
positive_list = pd.DataFrame(positive_list)
print("total number: ",len(tweet_list_2))
print("positive number: ",len(positive_list))
print("negative number: ", len(negative_list))
print("neutral number: ",len(neutral_list))

total number:  10
positive number:  0
negative number:  0
neutral number:  10


In [ ]:
# tweet_list.to_csv("tweet_list.csv", index=False, sep=";")

In [17]:
df_export = pd.DataFrame({"Tweet":tweet_list, "Score":score_list, "Date": date_list, "User": user_list, "Location": location_list})

In [18]:
#df_export.to_csv("df_export.csv", index=False, sep=";", encoding="utf-8")
df_export.to_excel("df_export.xlsx")

In [19]:
df_export

,Tweet,Score,Date,User,Location
0,"Aprovecha esta beca para irte a ""la Terreta"".✈...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",2022-11-16 12:26:50,Santander SmartBank,
1,El @comite_economic otorga uno de los Premios ...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",2022-11-16 12:19:23,Comité Econòmic i Social Comunitat Valenciana,Castelló
2,«Rata colonizadora»: el asedio a una joven de ...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",2022-11-16 12:00:17,jalocu1960,Bilbao
3,RT @HablamosE: Los talibanes lingüísticos llam...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",2022-11-16 11:55:11,jalocu1960,Bilbao
4,RT @viceacademicouc: 130 años de la CREACIÓN D...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",2022-11-16 11:40:28,Reina X. Ferrer D.,"Valencia, Venezuela"
5,RT @EduarsanchezUc: #UnDiaComoHoy el 15 de nov...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",2022-11-16 11:38:23,Reina X. Ferrer D.,"Valencia, Venezuela"
6,RT @EduarsanchezUc: #UnDiaComoHoy el 15 de nov...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",2022-11-16 11:13:16,Pablo Zabaleta Polo 🇻🇪 🇺🇦,Venezuela
7,RT @RevistaRETEMA: Arranca en #Valencia una nu...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",2022-11-16 11:05:18,YWP Spanish Chapter,
8,RT @PsicopirE: #Webinar sobre #PrevenciónSuici...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",2022-11-16 11:00:11,Edurne Crespo,
9,Arranca en #Valencia una nueva edición del Con...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",2022-11-16 11:00:03,"RETEMA, Revista Técnica de Medio Ambiente",Madrid


In [22]:
tweet


Status(_api=<tweepy.api.API object at 0x7fb8aac55710>, _json={'created_at': 'Wed Nov 16 11:00:03 +0000 2022', 'id': 1592834879233806336, 'id_str': '1592834879233806336', 'text': 'Arranca en #Valencia una nueva edición del Congreso IWA @YWP_Spain 2022.\n\n➕ El evento se celebra entre el 16 y el 1… https://t.co/pyvIeh6ofS', 'truncated': True, 'entities': {'hashtags': [{'text': 'Valencia', 'indices': [11, 20]}], 'symbols': [], 'user_mentions': [{'screen_name': 'YWP_Spain', 'name': 'YWP Spanish Chapter', 'id': 700646113410359296, 'id_str': '700646113410359296', 'indices': [56, 66]}], 'urls': [{'url': 'https://t.co/pyvIeh6ofS', 'expanded_url': 'https://twitter.com/i/web/status/1592834879233806336', 'display_url': 'twitter.com/i/web/status/1…', 'indices': [117, 140]}]}, 'metadata': {'iso_language_code': 'es', 'result_type': 'recent'}, 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_repl